In [131]:
from importlib import reload
import numpy as np
import helperfuncs as hp
import ldsc_72 as ld
import matplotlib.pyplot as plt
import scipy.stats
reload(ld)

<module 'ldsc_72' from 'C:\\Users\\Hariharan\\Documents\\git_repos\\SNIPar\\ldsc_reg\\infertheta\\ldsc_72.py'>

In [132]:
# Getting model set up

np.random.seed(123)
N = 100
S_size=  int(N/2)
S = np.array([100/N] * N).reshape((N, 1, 1))
V = np.identity(1) * 100

model = ld.sibreg(S = S)
model.simdata(V, N, simr = True)

Warning there is no value for theta. Maybe consider simulating it
No value for U given. Generating a vector of ones (all SNPs weighted equally)
No value for r given. Generating a vector of ones for r
Simulated LD scores!
Effect Vectors Simulated!


In [133]:
negll, neggrad = model.neg_logll_grad(V)

TypeError: cannot unpack non-iterable numpy.float64 object

In [127]:
neggrad

array([-34.79564052])

In [134]:
model.solve(est_init = np.atleast_2d(0))

(array([[464.68310536]]),       fun: 274.63745118919684
  hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
       jac: array([5.68434189e-06])
   message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
      nfev: 36
       nit: 16
    status: 0
   success: True
         x: array([464.68310536]))

In [136]:
def logll(V):

    Vin = hp.extract_upper_triangle(V)
    logll = -model.neg_logll_grad(Vin) # [0]
    
    return logll

def gradlogll(V):

    Vin = hp.extract_upper_triangle(V)
    grad = -model.neg_logll_grad(Vin)[1]
    
    return grad

def num_grad(V, row = 0, col = 0, dx = 1e-6):
    
    V1 = np.copy(V)
    V1[row, col] += dx
    
    derivative = (logll(V1) - logll(V))/dx
    
    return derivative

num_grad(V)

0.9877539923763834

In [130]:
-model.neg_logll_grad(V)[1][0]

34.795640523907664

Comparing Log Likelihood Functions

In [78]:
# comparing log likelihoods

Si = S[0]
ri = model.r[0]
logll = scipy.stats.multivariate_normal.logpdf(model.theta, 
                                               mean = np.zeros(1),
                                               cov = Si + ri * V/N)
logll.sum()

-70.66271854798362

In [79]:
-model.neg_logll_grad(V)[0]

-62.904707638511326